In [ ]:
import pandas as pd
from pystan import StanModel
import pickle
from matplotlib.figure import figaspect

In [ ]:
loserwinner = pd.read_csv('output/data_kaimei.csv', index_col=0)

In [ ]:
loserwinner = loserwinner[loserwinner['year'] > 2012]
loserwinner = loserwinner.reset_index(drop=True)

In [ ]:
#loserwinner

In [ ]:
len(loserwinner['loser'].unique())

In [ ]:
len(loserwinner['loser'].value_counts()[loserwinner['loser'].value_counts()<5])

In [ ]:
len(loserwinner['winner'].value_counts()[loserwinner['winner'].value_counts()<5])

In [ ]:
rare_rikishies = []
rare_rikishies.append(loserwinner['loser'].value_counts()[loserwinner['loser'].value_counts()<5].index)
rare_rikishies.append(loserwinner['winner'].value_counts()[loserwinner['winner'].value_counts()<5].index)
rare_rikishies = [rare_rikishi for loser_winner in rare_rikishies for rare_rikishi in loser_winner]

In [ ]:
for rare_rikishi in rare_rikishies:
    loserwinner = loserwinner[loserwinner['loser'] != rare_rikishi]
    loserwinner = loserwinner[loserwinner['winner'] != rare_rikishi]
loserwinner = loserwinner.reset_index(drop=True)

In [ ]:
#loserwinner

In [ ]:
len(loserwinner['loser'].unique())

In [ ]:
len(loserwinner['winner'].unique())

In [ ]:
rikishi_id = dict(zip(loserwinner['loser'].unique(), range(1, len(loserwinner['loser'].unique())+1)))

In [ ]:
#rikishi_id

In [ ]:
loserwinner = loserwinner.replace(rikishi_id)

In [ ]:
loserwinner['year'] = loserwinner['year']-2012

In [ ]:
#loserwinner

In [ ]:
loserwinner.info()

In [ ]:
loserwinner['loser'].min()

In [ ]:
loserwinner['loser'].max()

In [ ]:
N = loserwinner['loser'].max()
M = loserwinner.shape[0]
L = loserwinner['year'].max()
Id = list(loserwinner[['loser', 'winner', 'year']].values)

data = dict(
    N=N,
    M=M,
    L=L,
    Id=Id
)

In [ ]:
model = '''
data {
    int N;
    int M;
    int L;
    int<lower=1, upper=N> Id[M, 3];
}

parameters {
    ordered[2] performance[M];
    matrix[L, N-1] strength0;
    real<lower=0> s_strength;
    real<lower=0> s_time_strength;
    vector<lower=0>[N] stability;
}

transformed parameters {
    matrix[L, N] strength;
    strength[, 1:N-1] = strength0;
    for (i in 1:L)
        strength[i, N] = -sum(strength0[i, ]);
}

model {
    strength[1, ] ~ normal(0, s_strength);
    for (k in 2:L)
        strength[k, ] ~ normal(strength[k-1, ], s_time_strength);
    stability ~ gamma(10, 10);
    for (i in 1:M)
        for (j in 1:2)
            performance[i, j] ~ student_t(1, strength[Id[i, 3], Id[i, j]], stability[Id[i, j]]);
}
'''

In [ ]:
stanmodel = StanModel(model_code=model)

In [ ]:
# ADVI (Automatic Differentiation Variational Inference)
fit_vb = stanmodel.vb(data=data, seed=123)
#about 30s

In [ ]:
#with open('5years_advi_timex.pkl', 'wb') as f:
#    pickle.dump(stanmodel, f)
#    pickle.dump(fit_vb, f)

In [ ]:
#fit_vb.keys()

In [ ]:
vb_sample = pd.read_csv(fit_vb['args']['sample_file'].decode('utf-8'), comment='#')
vb_sample = vb_sample.drop([0,1])

In [ ]:
np.mean(vb_sample.filter(regex='s_strength'))

In [ ]:
np.mean(vb_sample.filter(regex='s_time_strength'))

In [ ]:
strength = vb_sample.filter(regex='strength\.\d+')

In [ ]:
#strength

In [ ]:
strength1 = vb_sample.filter(regex='strength\.1\.\d+')
strength2 = vb_sample.filter(regex='strength\.2\.\d+')
strength3 = vb_sample.filter(regex='strength\.3\.\d+')
strength4 = vb_sample.filter(regex='strength\.4\.\d+')
strength5 = vb_sample.filter(regex='strength\.5\.\d+')
strength6 = vb_sample.filter(regex='strength\.6\.\d+')

In [ ]:
stability = vb_sample.filter(regex='stability\.\d+')

In [ ]:
#strength1

In [ ]:
#stability

In [ ]:
rikishi_df = pd.DataFrame(index=range(1, len(rikishi_id)+1), columns=['strength1', 'strength2', 'strength3', 'strength4', 'strength5', 'strength6'])

In [ ]:
rikishi_df['strength1'] = np.mean(strength1).values
rikishi_df['strength2'] = np.mean(strength2).values
rikishi_df['strength3'] = np.mean(strength3).values
rikishi_df['strength4'] = np.mean(strength4).values
rikishi_df['strength5'] = np.mean(strength5).values
rikishi_df['strength6'] = np.mean(strength6).values

In [ ]:
rikishi_df['id'] = rikishi_df.index

In [ ]:
rikishi_df = rikishi_df.sort_values(by='strength6', ascending=False)

In [ ]:
rikishi_df['rank'] = range(1, len(rikishi_df)+1)

In [ ]:
rikishi_id_inverse = dict(zip(rikishi_id.values(), rikishi_id.keys()))

In [ ]:
rikishi_df.index = rikishi_df['id'].replace(rikishi_id_inverse).values

In [ ]:
rikishi_df

In [ ]:
rikishi_df.describe()

In [ ]:
strong = pd.DataFrame(data=[rikishi_df['rank'].values, rikishi_df['id'].values, rikishi_df.index.values],
#                      index=['rank', 'id', 'name']).T.iloc[:10, :]
                      index=['rank', 'id', 'name']).T

In [ ]:
#sns.distplot(rikishi_df['strength1'])

In [ ]:
#sns.distplot(rikishi_df['strength4'])

In [ ]:
#rikishi_df.sort_values(by='strength4', ascending=False).T.iloc[:, :10].plot(figsize=(10, 6))

In [ ]:
#strength

In [ ]:
strong

In [ ]:
probs = (10, 25, 50, 75, 90)
cols = ['p{}'.format(p) for p in probs]
cols.append('x')

In [ ]:
transitions = pd.Panel(major_axis=range(L), minor_axis=cols)

In [ ]:
for i in range(len(strong)):
    id = strong['id'][i]
    transition = pd.DataFrame(np.percentile(strength.T[strength.columns.str.endswith(f'.{id}')].T, probs, axis=0)).T
    transition.columns = ['p{}'.format(p) for p in probs]
    transition['x'] = transition.index + 2013
    transitions[i] = transition

In [ ]:
transitions[1]

In [ ]:
plt.figure(figsize=(10, 6))
ax = plt.axes()
cmap = plt.cm.get_cmap('tab10')

for i in range(len(strong)):
#for i in range(65, 72):
    c = cmap(i%10)
    ax.plot('x', 'p50', data=transitions[i], color=c)
#    ax.fill_between('x', 'p10', 'p90', data=transitions[i], color=c, alpha=0.1)
#    ax.fill_between('x', 'p25', 'p75', data=transitions[i], color=c, alpha=0.2)
ax.legend(strong['name'])

plt.setp(ax, xlabel='year', ylabel='strength')
plt.xticks(range(2013, 2019))
plt.show()
plt.savefig('5years_advi_time', dpi=200)

明生の取り組みが2018年しかなくて、計算で求まる2018のstrengthに無理やり2017から合わせるから、それに合わせてs_strength(s_time_strength)が決まってしまう。

In [ ]:
check = loserwinner.groupby(['loser', 'year']).sum().index.values

In [ ]:
#check

In [ ]:
#check_df = pd.DataFrame(index=range(1, 1+loserwinner['loser'].max()), columns=['1', '2', '3', '4'])
check_df = pd.DataFrame(index=range(1, 1+loserwinner['loser'].max()))
for i in range(len(check)):
    rikishi = check[i][0]
    year = check[i][1]-1
    check_df.loc[rikishi, year] = True

In [ ]:
check_df

In [ ]:
id_rank = dict(zip(strong['id'], strong['rank']))

In [ ]:
for id in range(1, 1+len(strong)):
    rank = id_rank[id]
    for j in range(check_df.shape[1]):
        if check_df.loc[id, j] != True:
            transitions[rank-1].loc[j] = np.nan

In [ ]:
plt.figure(figsize=(10, 6))
ax = plt.axes()
cmap = plt.cm.get_cmap('tab10')

#for i in range(len(strong)):
for i in range(10):
    c = cmap(i%10)
    ax.plot('x', 'p50', data=transitions[i], color=c, marker='o')
#    ax.fill_between('x', 'p10', 'p90', data=transitions[i], color=c, alpha=0.1)
#    ax.fill_between('x', 'p25', 'p75', data=transitions[i], color=c, alpha=0.2)
ax.legend(strong['name'], loc=2)

plt.setp(ax, xlabel='year', ylabel='strength')
plt.xticks(range(2013, 2019))
plt.show()
plt.savefig('5years_advi_time', dpi=200)